In [ ]:
import socket
import threading
import json

class Client:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.client_socket = None
        self.message_queue = []
        self.expected_sequence = 0  

    def connect(self):
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((self.host, self.port))
        print(f"Connected to server at {self.host}:{self.port}")

        threading.Thread(target=self.receive_messages, daemon=True).start()

    def receive_messages(self):
        while True:
            try:
                data = self.client_socket.recv(1024).decode()
                if not data:
                    break

                message = json.loads(data)
                print(f"Received message: {message}")
                self.deliver_message(message)
            except Exception as e:
                print(f"Error receiving message: {e}")
                break

    def deliver_message(self, message):
        seq = message['sequence']
        if seq == self.expected_sequence:
            print(f"Delivered: {message['content']} (Sequence: {seq})")
            self.expected_sequence += 1
        else:
            self.message_queue.append(message)
            self.message_queue.sort(key=lambda x: x['sequence'])

            # Try delivering from the queue
            while self.message_queue and self.message_queue[0]['sequence'] == self.expected_sequence:
                next_msg = self.message_queue.pop(0)
                print(f"Delivered from queue: {next_msg['content']} (Sequence: {next_msg['sequence']})")
                self.expected_sequence += 1

    def send_message(self, content):
        message = {'content': content}
        self.client_socket.sendall(json.dumps(message).encode())

if __name__ == "__main__":
    client = Client("192.168.122.221", 8080)
    try:
        client.connect()
        while True:
            msg = input("Enter message (type 'exit' to disconnect): ")
            if msg.lower() == 'exit':
                break
            client.send_message(msg)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        print("Client disconnected.")


Connected to server at 192.168.122.221:8080
Received message: {'content': 'hi', 'sequence': 1}
Delivered: hi (Sequence: 1)
Received message: {'content': 'im here', 'sequence': 2}
Delivered: im here (Sequence: 2)
Client disconnected.
